### Read txt files

In [1]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
import h5py
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 

image_base_path = 'nabirds/images/'

In [23]:
images = "nabirds/images.txt"
train_test_split = "nabirds/train_test_split.txt"
classes = "nabirds/classes.txt"
image_class_labels = "nabirds/image_class_labels.txt"

def load_txt():
    img_dic = {}
    train_dic = {}
    class_dic = {}
    image_class_dic = {}
    
    with open(images) as img_object:
        for line in img_object:
            vals = line.replace('\n', '').split(' ')
            img_dic[vals[0]] = vals[1]
            
    with open(train_test_split) as train_object:
        for line in train_object:
            vals = line.replace('\n', '').split(' ')
            train_dic[vals[0]] = vals[1]
            
    with open(classes) as class_object:
        for line in class_object:
            vals = line.replace('\n', '').split(' ')
            key = vals[0]
            vals.remove(key)
            val = ' '.join(vals)
            class_dic[key] = (val)
            
    with open(image_class_labels) as image_class_object:
        for line in image_class_object:
            vals = line.replace('\n', '').split(' ')
            image_class_dic[vals[0]] = vals[1]
    
    image_infos = []
    for key in img_dic:
        location = img_dic[key]
        isTraining = bool(int(train_dic[key]))
        class_name = class_dic[image_class_dic[key]]
        image_infos.append([location, isTraining, class_name])
    return image_infos

def getTestInfo():
    data = [d for d in load_txt() if d[1] == True]
    return data

def getTrainInfo():
    data = [d for d in load_txt() if d[1] == False]
    return data

In [24]:
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(224,224,3)
    
#     img = img.astype('float32')  
#     img = img - [123.68, 116.779, 103.939]  
    return img

In [25]:
train_info = getTrainInfo()

def load_images(infos):
    images = []
    i = 0
    for info in infos:
        if(i < 100):
            images.append(load_image(image_base_path + info[0]))
        else:
            break
        i = i + 1
    return np.array(images)

targets = [[target[2]] for target in train_info]

targets = [targets[i] for i in range(100)]

images = load_images(train_info)

encoder = OneHotEncoder()
targets = np.array(encoder.fit_transform(targets).toarray())




In [41]:
# define cnn model 
def define_model():  
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(94, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# run the test harness for evaluating a model
def run_test_harness():
# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
#     featurewise_center=True
	# specify imagenet mean values for centering
# 	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow(images, targets)
	# fit model
	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=1)
	# save model
	model.save('final_model.h5')
 
# entry point, run the test harness
run_test_harness()

Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 5.2042 - accuracy: 0.0227
Epoch 2/10
4/4 [==============================] - 11s 2s/step - loss: 5.1819 - accuracy: 0.0344
Epoch 3/10
4/4 [==============================] - 10s 2s/step - loss: 5.0752 - accuracy: 0.0069
Epoch 4/10
4/4 [==============================] - 11s 2s/step - loss: 4.7728 - accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 11s 2s/step - loss: 4.4132 - accuracy: 0.0464
Epoch 6/10
4/4 [==============================] - 10s 2s/step - loss: 4.3460 - accuracy: 0.0615
Epoch 7/10
4/4 [==============================] - 10s 3s/step - loss: 4.5989 - accuracy: 0.1134
Epoch 8/10
4/4 [==============================] - 10s 3s/step - loss: 4.5847 - accuracy: 0.0194
Epoch 9/10
4/4 [==============================] - 11s 2s/step - loss: 4.4275 - accuracy: 0.0221
Epoch 10/10
4/4 [==============================] - 10s 2s/step - loss: 4.1489 - accuracy: 0.0257


In [4]:
# make a prediction for a new image.
classes = ["Abert's Towhee", 'Acorn Woodpecker',
       "Allen's Hummingbird (Adult Male)",
       "Allen's Hummingbird (Female, immature)", 'American Avocet',
       'American Black Duck', 'American Coot', 'American Crow',
       'American Dipper', 'American Goldfinch (Breeding Male)',
       'American Goldfinch (Female/Nonbreeding Male)',
       'American Kestrel (Adult male)',
       'American Kestrel (Female, immature)', 'American Oystercatcher',
       'American Pipit', 'American Redstart (Adult Male)',
       'American Redstart (Female/juvenile)', 'American Robin (Adult)',
       'American Robin (Juvenile)', 'American Tree Sparrow',
       'American White Pelican', 'American Wigeon (Breeding male)',
       'American Wigeon (Female/Eclipse male)', 'American Woodcock',
       'Anhinga', "Anna's Hummingbird (Adult Male)",
       "Anna's Hummingbird (Female, immature)", 'Ash-throated Flycatcher',
       'Bald Eagle (Adult, subadult)', 'Bald Eagle (Immature, juvenile)',
       'Baltimore Oriole (Adult male)',
       'Baltimore Oriole (Female/Immature male)', 'Band-tailed Pigeon',
       'Bank Swallow', 'Barn Owl', 'Barn Swallow', 'Barred Owl',
       "Barrow's Goldeneye (Breeding male)",
       "Barrow's Goldeneye (Female/Eclipse male)",
       'Bay-breasted Warbler (Breeding male)',
       'Bay-breasted Warbler (Female, Nonbreeding male, Immature)',
       "Bell's Vireo", 'Belted Kingfisher', "Bewick's Wren",
       'Black Guillemot (Breeding)',
       'Black Guillemot (Nonbreeding, juvenile)', 'Black Oystercatcher',
       'Black Phoebe', 'Black Rosy-Finch',
       'Black Scoter (Female/juvenile)', 'Black Scoter (Male)',
       'Black Skimmer', 'Black Tern', 'Black Turnstone', 'Black Vulture',
       'Black-and-white Warbler', 'Black-bellied Plover (Breeding)',
       'Black-bellied Plover (Nonbreeding/juvenile)',
       'Black-bellied Whistling-Duck', 'Black-billed Cuckoo',
       'Black-billed Magpie', 'Black-capped Chickadee',
       'Black-chinned Hummingbird (Adult Male)',
       'Black-chinned Hummingbird (Female, immature)',
       'Black-crested Titmouse', 'Black-crowned Night-Heron (Adult)',
       'Black-crowned Night-Heron (Immature)',
       'Black-headed Grosbeak (Adult Male)',
       'Black-headed Grosbeak (Female/immature male)',
       'Black-legged Kittiwake (Adult)',
       'Black-legged Kittiwake (Immature)', 'Black-necked Stilt',
       'Black-tailed Gnatcatcher',
       'Black-throated Blue Warbler (Adult Male)',
       'Black-throated Blue Warbler (Female/Immature male)',
       'Black-throated Gray Warbler', 'Black-throated Green Warbler',
       'Blackburnian Warbler', 'Blackpoll Warbler (Breeding male)',
       'Blackpoll Warbler (Female/juvenile)',
       'Blue Grosbeak (Adult Male)', 'Blue Grosbeak (Female/juvenile)',
       'Blue Jay', 'Blue-gray Gnatcatcher', 'Blue-headed Vireo',
       'Blue-winged Teal  (Female/juvenile)', 'Blue-winged Teal  (Male)',
       'Blue-winged Warbler', 'Boat-tailed Grackle',
       'Bobolink (Breeding male)',
       'Bobolink (Female/juvenile/nonbreeding male)', 'Bohemian Waxwing',
       "Bonaparte's Gull", 'Boreal Chickadee', "Brandt's Cormorant",
       'Brant', "Brewer's Blackbird (Female/Juvenile)",
       "Brewer's Blackbird (Male)", "Brewer's Sparrow",
       'Bridled Titmouse', 'Broad-billed Hummingbird (Adult Male)',
       'Broad-billed Hummingbird (Female, immature)',
       'Broad-tailed Hummingbird (Adult Male)',
       'Broad-tailed Hummingbird (Female, immature)',
       'Broad-winged Hawk (Adult)', 'Broad-winged Hawk (Immature)',
       'Bronzed Cowbird', 'Brown Creeper', 'Brown Pelican',
       'Brown Thrasher', 'Brown-capped Rosy-Finch',
       'Brown-headed Cowbird (Female/Juvenile)',
       'Brown-headed Cowbird (Male)', 'Brown-headed Nuthatch',
       'Bufflehead (Breeding male)', 'Bufflehead (Female/immature male)',
       "Bullock's Oriole (Adult male)",
       "Bullock's Oriole (Female/Immature male)", 'Burrowing Owl',
       'Bushtit', 'Cackling Goose', 'Cactus Wren',
       'California Gull (Adult)', 'California Gull (Immature)',
       'California Quail (Female/juvenile)', 'California Quail (Male)',
       'California Thrasher', 'California Towhee',
       'Calliope Hummingbird (Adult Male)',
       'Calliope Hummingbird (Female, immature)', 'Canada Goose',
       'Canada Warbler', 'Canvasback (Breeding male)',
       'Canvasback (Female/Eclipse male)', 'Canyon Towhee', 'Canyon Wren',
       'Cape May Warbler', 'Carolina Chickadee', 'Carolina Wren',
       'Caspian Tern', "Cassin's Finch (Adult Male)",
       "Cassin's Finch (Female/immature)", "Cassin's Kingbird",
       "Cassin's Vireo", 'Cattle Egret', 'Cave Swallow', 'Cedar Waxwing',
       'Chestnut-backed Chickadee',
       'Chestnut-sided Warbler (Breeding male)',
       'Chestnut-sided Warbler (Female/immature male)',
       'Chihuahuan Raven', 'Chimney Swift', 'Chipping Sparrow (Breeding)',
       'Chipping Sparrow (Immature/nonbreeding adult)',
       'Cinnamon Teal (Female/juvenile)', 'Cinnamon Teal (Male)',
       "Clark's Grebe", "Clark's Nutcracker", 'Clay-colored Sparrow',
       'Cliff Swallow', 'Common Eider (Adult male)',
       'Common Eider (Female/juvenile)',
       'Common Eider (Immature/Eclipse male)', 'Common Gallinule (Adult)',
       'Common Gallinule (Immature)', 'Common Goldeneye (Breeding male)',
       'Common Goldeneye (Female/Eclipse male)', 'Common Grackle',
       'Common Ground-Dove', 'Common Loon (Breeding)',
       'Common Loon (Nonbreeding/juvenile)',
       'Common Merganser (Breeding male)',
       'Common Merganser (Female/immature male)', 'Common Nighthawk',
       'Common Raven', 'Common Redpoll', 'Common Tern',
       'Common Yellowthroat (Adult Male)',
       'Common Yellowthroat (Female/immature male)',
       "Cooper's Hawk (Adult)", "Cooper's Hawk (Immature)",
       'Cordilleran Flycatcher', "Costa's Hummingbird (Adult Male)",
       "Costa's Hummingbird (Female, immature)", 'Crested Caracara',
       'Curve-billed Thrasher', 'Dark-eyed Junco (Oregon)',
       'Dark-eyed Junco (Pink-sided)',
       'Dark-eyed Junco (Red-backed/Gray-headed)',
       'Dark-eyed Junco (Slate-colored)',
       'Dark-eyed Junco (White-winged)', 'Dickcissel',
       'Double-crested Cormorant (Adult)',
       'Double-crested Cormorant (Immature)', 'Downy Woodpecker',
       'Dunlin (Breeding)', 'Dunlin (Nonbreeding/juvenile)',
       'Eared Grebe (Breeding)', 'Eared Grebe (Nonbreeding/juvenile)',
       'Eastern Bluebird', 'Eastern Kingbird', 'Eastern Meadowlark',
       'Eastern Phoebe', 'Eastern Screech-Owl', 'Eastern Towhee',
       'Eastern Wood-Pewee', 'Eurasian Collared-Dove',
       'European Starling (Breeding Adult)',
       'European Starling (Juvenile)',
       'European Starling (Nonbreeding Adult)',
       'Evening Grosbeak (Adult Male)',
       'Evening Grosbeak (Female/Juvenile)', 'Field Sparrow', 'Fish Crow',
       'Florida Scrub-Jay', "Forster's Tern", 'Fox Sparrow (Red)',
       'Fox Sparrow (Sooty)', 'Fox Sparrow (Thick-billed/Slate-colored)',
       'Gadwall (Breeding male)', 'Gadwall (Female/Eclipse male)',
       "Gambel's Quail (Female/juvenile)", "Gambel's Quail (Male)",
       'Gila Woodpecker', 'Glaucous-winged Gull (Adult)',
       'Glaucous-winged Gull (Immature)', 'Glossy Ibis',
       'Golden Eagle (Adult)', 'Golden Eagle (Immature)',
       'Golden-crowned Kinglet', 'Golden-crowned Sparrow (Adult)',
       'Golden-crowned Sparrow (Immature)', 'Golden-fronted Woodpecker',
       'Gray Catbird', 'Gray Jay', 'Gray-crowned Rosy-Finch',
       'Great Black-backed Gull (Adult)',
       'Great Black-backed Gull (Immature)', 'Great Blue Heron',
       'Great Cormorant (Adult)', 'Great Cormorant (Immature)',
       'Great Crested Flycatcher', 'Great Egret', 'Great Horned Owl',
       'Great-tailed Grackle', 'Greater Roadrunner',
       'Greater Scaup (Breeding male)',
       'Greater Scaup (Female/Eclipse male)',
       'Greater White-fronted Goose', 'Greater Yellowlegs', 'Green Heron',
       'Green-tailed Towhee', 'Green-winged Teal  (Female/juvenile)',
       'Green-winged Teal (Male)', 'Hairy Woodpecker',
       'Harlequin Duck (Female/juvenile)', 'Harlequin Duck (Male)',
       "Harris's Hawk", "Harris's Sparrow (Adult)",
       "Harris's Sparrow (Immature)", "Heermann's Gull (Adult)",
       "Heermann's Gull (Immature)", 'Hermit Thrush', 'Hermit Warbler',
       'Herring Gull (Adult)', 'Herring Gull (Immature)', 'Hoary Redpoll',
       'Hooded Merganser (Breeding male)',
       'Hooded Merganser (Female/immature male)',
       'Hooded Oriole (Adult male)',
       'Hooded Oriole (Female/Immature male)', 'Hooded Warbler',
       'Horned Grebe (Breeding)', 'Horned Grebe (Nonbreeding/juvenile)',
       'Horned Lark', 'House Finch (Adult Male)',
       'House Finch (Female/immature)', 'House Sparrow (Female/Juvenile)',
       'House Sparrow (Male)', 'House Wren', "Hutton's Vireo",
       'Inca Dove', 'Indigo Bunting (Adult Male)',
       'Indigo Bunting (Female/juvenile)', 'Juniper Titmouse', 'Killdeer',
       'Ladder-backed Woodpecker', 'Lark Bunting (Breeding male)',
       'Lark Bunting (Female/Nonbreeding male)', 'Lark Sparrow',
       'Laughing Gull (Breeding)', 'Laughing Gull (Nonbreeding/Immature)',
       'Lazuli Bunting (Adult Male)', 'Lazuli Bunting (Female/juvenile)',
       'Least Flycatcher', 'Least Sandpiper',
       'Lesser Goldfinch (Adult Male)',
       'Lesser Goldfinch (Female/juvenile)',
       'Lesser Scaup (Breeding male)',
       'Lesser Scaup (Female/Eclipse male)', 'Lesser Yellowlegs',
       "Lincoln's Sparrow", 'Little Blue Heron (Adult)',
       'Little Blue Heron (Immature)', 'Loggerhead Shrike',
       'Long-billed Curlew', 'Long-tailed Duck (Female/juvenile)',
       'Long-tailed Duck (Summer male)', 'Long-tailed Duck (Winter male)',
       'Louisiana Waterthrush', "MacGillivray's Warbler",
       'Magnolia Warbler (Breeding male)',
       'Magnolia Warbler (Female/immature male)',
       'Mallard (Breeding male)', 'Mallard (Female/Eclipse male)',
       'Marbled Godwit', 'Marsh Wren', 'Merlin', 'Mew Gull',
       'Mexican Jay', 'Mississippi Kite', 'Monk Parakeet', 'Mottled Duck',
       'Mountain Bluebird', 'Mountain Chickadee', 'Mourning Dove',
       'Mourning Warbler', 'Mute Swan', 'Nashville Warbler',
       'Neotropic Cormorant', 'Northern Bobwhite',
       'Northern Cardinal (Adult Male)',
       'Northern Cardinal (Female/Juvenile)',
       'Northern Flicker (Red-shafted)',
       'Northern Flicker (Yellow-shafted)',
       'Northern Gannet (Adult, Subadult)',
       'Northern Gannet (Immature/Juvenile)',
       'Northern Harrier (Adult male)',
       'Northern Harrier (Female, immature)', 'Northern Mockingbird',
       'Northern Parula', 'Northern Pintail (Breeding male)',
       'Northern Pintail (Female/Eclipse male)', 'Northern Pygmy-Owl',
       'Northern Rough-winged Swallow', 'Northern Saw-whet Owl',
       'Northern Shoveler (Breeding male)',
       'Northern Shoveler (Female/Eclipse male)', 'Northern Shrike',
       'Northern Waterthrush', 'Northwestern Crow',
       "Nuttall's Woodpecker", 'Oak Titmouse', 'Orange-crowned Warbler',
       'Orchard Oriole (Adult Male)', 'Orchard Oriole (Female/Juvenile)',
       'Orchard Oriole (Immature Male)', 'Osprey', 'Ovenbird',
       'Pacific Loon (Breeding)', 'Pacific Loon (Nonbreeding/juvenile)',
       'Pacific Wren', 'Pacific-slope Flycatcher',
       'Painted Bunting (Adult Male)',
       'Painted Bunting (Female/juvenile)', 'Palm Warbler',
       'Pelagic Cormorant', 'Peregrine Falcon (Adult)',
       'Peregrine Falcon (Immature)', 'Phainopepla (Female/juvenile)',
       'Phainopepla (Male)', 'Pied-billed Grebe',
       'Pigeon Guillemot (Breeding)',
       'Pigeon Guillemot (Nonbreeding, juvenile)', 'Pileated Woodpecker',
       'Pine Grosbeak (Adult Male)', 'Pine Grosbeak (Female/juvenile)',
       'Pine Siskin', 'Pine Warbler', 'Plumbeous Vireo', 'Prairie Falcon',
       'Prairie Warbler', 'Prothonotary Warbler',
       'Purple Finch (Adult Male)', 'Purple Finch (Female/immature)',
       'Purple Gallinule (Adult)', 'Purple Gallinule (Immature)',
       'Purple Martin (Adult male)', 'Purple Martin (Female/juvenile)',
       'Pygmy Nuthatch', 'Pyrrhuloxia', 'Red Crossbill (Adult Male)',
       'Red Crossbill (Female/juvenile)', 'Red-bellied Woodpecker',
       'Red-breasted Merganser (Breeding male)',
       'Red-breasted Merganser (Female/immature male)',
       'Red-breasted Nuthatch', 'Red-breasted Sapsucker',
       'Red-eyed Vireo', 'Red-headed Woodpecker (Adult)',
       'Red-headed Woodpecker (Immature)', 'Red-naped Sapsucker',
       'Red-necked Grebe (Breeding)',
       'Red-necked Grebe (Nonbreeding/juvenile)',
       'Red-shouldered Hawk (Adult )', 'Red-shouldered Hawk (Immature)',
       'Red-tailed Hawk (Dark morph)',
       'Red-tailed Hawk (Light morph adult)',
       'Red-tailed Hawk (Light morph immature)',
       'Red-throated Loon (Breeding)',
       'Red-throated Loon (Nonbreeding/juvenile)',
       'Red-winged Blackbird (Female/juvenile)',
       'Red-winged Blackbird (Male)', 'Reddish Egret (Dark morph)',
       'Reddish Egret (White morph)', 'Redhead (Breeding male)',
       'Redhead (Female/Eclipse male)', 'Ring-billed Gull (Adult)',
       'Ring-billed Gull (Immature)', 'Ring-necked Duck (Breeding male)',
       'Ring-necked Duck (Female/Eclipse male)',
       'Ring-necked Pheasant (Female/juvenile)',
       'Ring-necked Pheasant (Male)', 'Rock Pigeon',
       'Rose-breasted Grosbeak (Adult Male)',
       'Rose-breasted Grosbeak (Female/immature male)',
       'Roseate Spoonbill', "Ross's Goose",
       'Rough-legged Hawk (Dark morph)',
       'Rough-legged Hawk (Light morph)', 'Royal Tern',
       'Ruby-crowned Kinglet', 'Ruby-throated Hummingbird (Adult Male)',
       'Ruby-throated Hummingbird (Female, immature)',
       'Ruddy Duck (Breeding male)', 'Ruddy Duck (Female/juvenile)',
       'Ruddy Duck (Winter male)', 'Ruddy Turnstone', 'Ruffed Grouse',
       'Rufous Hummingbird (Adult Male)',
       'Rufous Hummingbird (Female, immature)', 'Rufous-crowned Sparrow',
       'Rusty Blackbird', 'Sanderling (Breeding)',
       'Sanderling (Nonbreeding/juvenile)', 'Sandhill Crane',
       'Savannah Sparrow', "Say's Phoebe", 'Scaled Quail',
       'Scarlet Tanager (Breeding Male)',
       'Scarlet Tanager (Female/Nonbreeding Male)',
       'Scissor-tailed Flycatcher', 'Semipalmated Plover',
       'Semipalmated Sandpiper', 'Sharp-shinned Hawk (Adult )',
       'Sharp-shinned Hawk (Immature)', 'Short-billed Dowitcher',
       'Snow Bunting (Breeding adult)', 'Snow Bunting (Nonbreeding)',
       'Snow Goose (Blue morph)', 'Snow Goose (White morph)',
       'Snowy Egret', 'Snowy Owl', 'Solitary Sandpiper', 'Song Sparrow',
       'Spotted Sandpiper (Breeding)',
       'Spotted Sandpiper (Nonbreeding/juvenile)', 'Spotted Towhee',
       "Steller's Jay", 'Summer Tanager (Adult Male)',
       'Summer Tanager (Female)', 'Summer Tanager (Immature Male)',
       'Surf Scoter (Female/immature)', 'Surf Scoter (Male)', 'Surfbird',
       "Swainson's Hawk (Dark morph )", "Swainson's Hawk (Immature)",
       "Swainson's Hawk (Light morph )", "Swainson's Thrush",
       'Swallow-tailed Kite', 'Swamp Sparrow', 'Tennessee Warbler',
       "Townsend's Solitaire", "Townsend's Warbler", 'Tree Swallow',
       'Tricolored Heron', 'Trumpeter Swan', 'Tufted Titmouse',
       'Tundra Swan', 'Turkey Vulture', 'Varied Thrush', "Vaux's Swift",
       'Veery', 'Verdin', 'Vermilion Flycatcher (Adult male)',
       'Vermilion Flycatcher (Female, immature)', 'Vesper Sparrow',
       'Violet-green Swallow', 'Warbling Vireo', 'Western Bluebird',
       'Western Grebe', 'Western Gull (Adult)', 'Western Gull (Immature)',
       'Western Kingbird', 'Western Meadowlark', 'Western Sandpiper',
       'Western Screech-Owl', 'Western Scrub-Jay',
       'Western Tanager (Breeding Male)',
       'Western Tanager (Female/Nonbreeding Male)', 'Western Wood-Pewee',
       'Whimbrel', 'White Ibis (Adult)', 'White Ibis (Immature)',
       'White-breasted Nuthatch', 'White-crowned Sparrow (Adult)',
       'White-crowned Sparrow (Immature)', 'White-eyed Vireo',
       'White-faced Ibis', 'White-tailed Kite',
       'White-throated Sparrow (Tan-striped/immature)',
       'White-throated Sparrow (White-striped)', 'White-throated Swift',
       'White-winged Crossbill (Adult Male)',
       'White-winged Crossbill (Female/juvenile)', 'White-winged Dove',
       'White-winged Scoter (Female/juvenile)',
       'White-winged Scoter (Male)', 'Wild Turkey', 'Willet',
       "Wilson's Phalarope (Breeding)",
       "Wilson's Phalarope (Nonbreeding, juvenile)", "Wilson's Snipe",
       "Wilson's Warbler", 'Winter Wren', 'Wood Duck (Breeding male)',
       'Wood Duck (Female/Eclipse male)', 'Wood Stork', 'Wood Thrush',
       'Wrentit', 'Yellow Warbler', 'Yellow-bellied Sapsucker',
       'Yellow-billed Cuckoo', 'Yellow-billed Magpie',
       'Yellow-breasted Chat', 'Yellow-crowned Night-Heron (Adult)',
       'Yellow-crowned Night-Heron (Immature)',
       'Yellow-headed Blackbird (Adult Male)',
       'Yellow-headed Blackbird (Female/Immature Male)',
       "Yellow-rumped Warbler (Breeding Audubon's)",
       'Yellow-rumped Warbler (Breeding Myrtle)',
       "Yellow-rumped Warbler (Winter/juvenile Audubon's)",
       'Yellow-rumped Warbler (Winter/juvenile Myrtle)',
       'Yellow-throated Vireo', 'Yellow-throated Warbler']
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(128, 128))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 128, 128, 3)
	# center pixel data
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('index.jpg')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result1 = model.predict_classes(img)
	print(classes[result1[0]])
	print(result1[0])    

	result2 = model.predict_proba(img)
	print(result2[:,result1])
 
# entry point, run the example
run_example()

White-crowned Sparrow (Adult)
515
[[0.9999999]]
